# GFK

In [1]:
%load_ext autoreload
%autoreload 2

In [19]:
#adaptacion
from utils.adaptacion import gfk_grid_search, transform_gfk

#clasificadores
from utils.clasificacion import *

#carga de datasets
from utils.DatasetStorage import Dataset
from utils.paths import *

#otros
import os
import numpy as np
import pandas as pd
from sklearn.externals import joblib

#variables para guardar los resultados
tipo = pruebas[2]
dataset_name = datasets[0]
dims = dimensions[dataset_name]

## Pruebas con el dataset Amazon

In [3]:
print tipo
print dataset_name
print dims
print data_path

gfk
amazon
1000
data


In [4]:
# cargando dataset Amazon
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
unlabeled = dataset_object.unlabeled
domains = dataset_object.domains

Dataset already splitted


# Adaptación

## Creación de modelos de adaptación.

Por cada par de dominios se entrenan distintos modelos según los parámetros establecidos.
Al entrenar cada modelo, este es inmediatamente probado, siendo almacenado o descartado según su desempeño.

Cada modelo es guardado en la ruta: models/amazon/gfk/me2\_[dominio\_fuente]_[dominio\_objetivo].pkl

### Creación de modelos de adaptación.

In [17]:
models_paths = os.path.join(models_path, dataset_name, tipo, 'me2_models_paths.pkl')

parameters = {
    'dims': [int(dims/4), int(dims/2)],
    'n_subs': [10, 20, 50]
}

#por cada par se entrena un adaptador y se guarda el que mejor resultados entrega
print "Adaptando entre:"
for src in domains:
    for tgt in domains:
        if src is not tgt:
            print "%s - %s" % (src, tgt)
            X_src = np.asarray(labeled[src]['X_tr'].todense())
            y_src = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)

            X_tgt = np.asarray(labeled[tgt]['X_tr'].todense())
            
            
            gfk, score = gfk_grid_search(X_src, y_src, X_tgt, parameters, n_jobs=4)
            print "\tMejor modelo: %.3f" % score
            
            folder_path = os.path.join(models_path, dataset_name, tipo)
            prefix = "me2_" + src + "_" + tgt + ".pkl"
            
            best_model_save_path = os.path.join(folder_path, prefix)
            print "\tGuardando modelo en %s" % best_model_save_path
            joblib.dump(gfk, best_model_save_path)

print "\nAdaptaciones completadas"

Adaptando entre:
kitchen - dvd
	n: 10 - d: 250 0.75375
	n: 20 - d: 250 0.738125
	n: 50 - d: 250 0.744375
	n: 10 - d: 500 0.75375
	n: 20 - d: 500 0.738125
	n: 50 - d: 500 0.744375
	Mejor modelo: 0.754
	Guardando modelo en models/amazon/gfk/me2_kitchen_dvd.pkl
kitchen - electronics
	n: 10 - d: 250 0.74625
	n: 20 - d: 250 0.74625
	n: 50 - d: 250 0.7475
	n: 10 - d: 500 0.74625
	n: 20 - d: 500 0.74625
	n: 50 - d: 500 0.7475
	Mejor modelo: 0.748
	Guardando modelo en models/amazon/gfk/me2_kitchen_electronics.pkl
kitchen - books
	n: 10 - d: 250 0.735
	n: 20 - d: 250 0.739375
	n: 50 - d: 250 0.7475
	n: 10 - d: 500 0.735
	n: 20 - d: 500 0.739375
	n: 50 - d: 500 0.7475
	Mejor modelo: 0.748
	Guardando modelo en models/amazon/gfk/me2_kitchen_books.pkl
dvd - kitchen
	n: 10 - d: 250 0.65875
	n: 20 - d: 250 0.689375
	n: 50 - d: 250 0.68375
	n: 10 - d: 500 0.65875
	n: 20 - d: 500 0.689375
	n: 50 - d: 500 0.68375
	Mejor modelo: 0.689
	Guardando modelo en models/amazon/gfk/me2_dvd_kitchen.pkl
dvd - elect

### Pruebas de clasificación

In [21]:
df = pd.DataFrame(columns=dataframe_columns)

i=1
tareas = len(domains)*(len(domains)-1)

# por cada par posible para adaptar
for src in domains:
    for tgt in domains:
        if src is not tgt:
            print "Tarea %d de %d" % (i, tareas)
            
            #baseline in-domain error
            #e_b(T,T)
            #entrenado en dominio tgt y probado en dominio tgt
            X_tr = np.asarray(labeled[tgt]['X_tr'].todense())
            y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)
            
            X_ts = np.asarray(labeled[tgt]['X_ts'].todense())
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            # se crean las rutas para cargar o crear los modelos
            model_name = "%s.pkl" % (tgt)
            model_path = os.path.join(models_path, dataset_name, "indomain", model_name)
            
            #Se realiza una clasificacion, estimando los parametros mediante cv
            svc = load_best_score(model_path, X_tr, y_tr)
            b_error = 1-svc.score(X_ts, y_ts)
            
            #############
            #### GFK ####
            #############
            # se adaptan los dominios usando GFK
            print "Adaptando dominios..."
            folder_path = os.path.join(models_path, dataset_name, tipo)
            prefix = "me2_" + src + "_" + tgt + ".pkl"
            
            best_model_save_path = os.path.join(folder_path, prefix)
            
            gfk = joblib.load(best_model_save_path)
            
            
            X_tr = np.asarray(labeled[src]['X_tr'].todense())
            X_tr_a = transform_gfk(X_tr, gfk)
            y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)
            
            X_ts = np.asarray(labeled[tgt]['X_ts'].todense())
            X_ts_a = transform_gfk(X_ts, gfk)
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            clf = get_best_score(X_tr_a, y_tr, classifier='SVC', n_jobs=4)
            t_error = 1-clf.score(X_ts_a, y_ts)
            
            
            # transfer loss t
            # t_error - b_error
            t_loss = t_error - b_error
            
            tarea = src[0]+'->'+tgt[0]
            df.loc[i] = ['GFK',tarea,src,tgt,b_error*100,t_error*100, t_loss*100]
            
            i += 1
            
print "\nPruebas completadas."

Tarea 1 de 12
Cargando modelo existente.
Adaptando dominios...


/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your d

Tarea 2 de 12
Cargando modelo existente.
Adaptando dominios...


/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your d

Tarea 3 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 4 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 5 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 6 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 7 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 8 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 9 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 10 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 11 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 12 de 12
Cargando modelo existente.
Adaptando dominios...

Pruebas completadas.


In [22]:
df

,Adaptacion,Tarea,Fuente,Objetivo,Baseline error,Transfer error,Transfer loss
1,GFK,k->d,kitchen,dvd,26.25,27.053176,0.803176
2,GFK,k->e,kitchen,electronics,23.50,13.940349,-9.559651
3,GFK,k->b,kitchen,books,25.50,31.510788,6.010788
4,GFK,d->k,dvd,kitchen,20.75,16.267907,-4.482093
5,GFK,d->e,dvd,electronics,23.50,23.883097,0.383097
6,GFK,d->b,dvd,books,25.50,24.768119,-0.731881
7,GFK,e->k,electronics,kitchen,20.75,11.777794,-8.972206
8,GFK,e->d,electronics,dvd,26.25,24.608115,-1.641885
9,GFK,e->b,electronics,books,25.50,31.475787,5.975787
10,GFK,b->k,books,kitchen,20.75,20.245506,-0.504494


In [23]:
new_scores_path = os.path.join(scores_path,dataset_name, tipo, "me2_%d.csv" % (dims))

print "Guardando en %s" % new_scores_path
df.to_csv(new_scores_path, columns=df.columns)
print "Resultados guardados."

Guardando en scores/amazon/gfk/me2_1000.csv
Resultados guardados.
